In [1]:
# %pip install langchain_community

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
myOpenAIapiKEY=os.getenv("OPENAI_API_KEY")

In [4]:
llm=ChatOpenAI(openai_api_key=myOpenAIapiKEY,model_name="gpt-3.5-turbo", temperature=0.7)

C:\Users\skong\AppData\Local\Temp\ipykernel_23516\2529314549.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(openai_api_key=myOpenAIapiKEY,model_name="gpt-3.5-turbo", temperature=0.7)


In [5]:
with open("D:\\Lab Setup\\MCQ Generator\\genAI\\response.json", 'r') as file:
    RESPONSE_JSON = json.load(file)

In [6]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
    )

In [9]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\skong\AppData\Local\Temp\ipykernel_23516\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [12]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [13]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [14]:
PATH = "D:/Lab Setup/MCQ Generator/genAI/data.txt"

In [15]:
with open(PATH, 'r') as f:
    TEXT = f.read()

In [16]:
print(TEXT)

Shivaji: Indian king

Shivaji born February 19 1630 or April 1627 Shivner Poona now Pune Indiadied April 3 1680 Raigad was an individual who opposed the Mughal dynasty and founded the Maratha kingdom in 17thcentury India His kingdoms security was based on religious toleration and on the functional integration of Brahmans Marathas and Prabhus

Early life and exploits.
Shivaji was descended from a line of prominent nobles India at the time of his birth in 1630 was under Muslim rule the Mughals in the north and the Muslim sultans of Bijapur and Golconda in the south All three ruled by right of conquest with no pretense that they had any obligations toward those who they ruled Shivaji whose ancestral estates were situated in the Deccan in the realm of the Bijapur sultans found the Muslim oppression and religious persecution of the Hindus so intolerable that by the time he was 16 he convinced himself that he was the divinely appointed instrument of the cause of Hindu freedoma conviction tha

In [17]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [18]:
NUMBER=5
SUBJECT="History"
TONE="simple"

In [19]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\skong\AppData\Local\Temp\ipykernel_23516\685591478.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Shivaji: Indian king

Shivaji born February 19 1630 or April 1627 Shivner Poona now Pune Indiadied April 3 1680 Raigad was an individual who opposed the Mughal dynasty and founded the Maratha kingdom in 17thcentury India His kingdoms security was based on religious toleration and on the functional integration of Brahmans Marathas and Prabhus

Early life and exploits.
Shivaji was descended from a line of prominent nobles India at the time of his birth in 1630 was under Muslim rule the Mughals in the north and the Muslim sultans of Bijapur and Golconda in the south All three ruled by right of conquest with no pretense that they had any obligations toward those who they ruled Shivaji whose ancestral estates were situated in the Deccan in the realm of the Bijapur sultans found the Muslim oppression and religious persecution of the Hindus so intolerable that by the time he was 16 he convinced himself that he was the divinely appointed instrument of the cause o

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for History students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{
    "1": {
        "mcq": "Who founded the Maratha kingdom in 17th century India?",
        "options": {
            "a": "Akbar",
            "b": "Shivaji",
            "c": "Aurangzeb",
            "d": "Humayun"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "Which battle turned the tide in favor of the Mughal dynasty in Hindustan?",
        "options": {
            "a": "First Battle of Panipat",
            "b": "Second Battle of Panipat",
            "c": "Ba

In [20]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:5595
Prompt Tokens:5140
Completion Tokens:455
Total Cost:0.00862


In [21]:
response

{'text': 'Shivaji: Indian king\n\nShivaji born February 19 1630 or April 1627 Shivner Poona now Pune Indiadied April 3 1680 Raigad was an individual who opposed the Mughal dynasty and founded the Maratha kingdom in 17thcentury India His kingdoms security was based on religious toleration and on the functional integration of Brahmans Marathas and Prabhus\n\nEarly life and exploits.\nShivaji was descended from a line of prominent nobles India at the time of his birth in 1630 was under Muslim rule the Mughals in the north and the Muslim sultans of Bijapur and Golconda in the south All three ruled by right of conquest with no pretense that they had any obligations toward those who they ruled Shivaji whose ancestral estates were situated in the Deccan in the realm of the Bijapur sultans found the Muslim oppression and religious persecution of the Hindus so intolerable that by the time he was 16 he convinced himself that he was the divinely appointed instrument of the cause of Hindu freedoma

In [22]:
quiz = response.get("quiz")

In [23]:
quiz

'{\n    "1": {\n        "mcq": "Who founded the Maratha kingdom in 17th century India?",\n        "options": {\n            "a": "Akbar",\n            "b": "Shivaji",\n            "c": "Aurangzeb",\n            "d": "Humayun"\n        },\n        "correct": "b"\n    },\n    "2": {\n        "mcq": "Which battle turned the tide in favor of the Mughal dynasty in Hindustan?",\n        "options": {\n            "a": "First Battle of Panipat",\n            "b": "Second Battle of Panipat",\n            "c": "Battle of Khanua",\n            "d": "Battle of Ghaghara"\n        },\n        "correct": "b"\n    },\n    "3": {\n        "mcq": "Which Mughal emperor is known for his policy of conciliation and conquest towards the Rajputs?",\n        "options": {\n            "a": "Akbar",\n            "b": "Shah Jahan",\n            "c": "Jahangir",\n            "d": "Babur"\n        },\n        "correct": "a"\n    },\n    "4": {\n        "mcq": "In which year did Shivaji have himself enthroned as an 

In [24]:
quiz=json.loads(quiz)

In [25]:
quiz

{'1': {'mcq': 'Who founded the Maratha kingdom in 17th century India?',
  'options': {'a': 'Akbar', 'b': 'Shivaji', 'c': 'Aurangzeb', 'd': 'Humayun'},
  'correct': 'b'},
 '2': {'mcq': 'Which battle turned the tide in favor of the Mughal dynasty in Hindustan?',
  'options': {'a': 'First Battle of Panipat',
   'b': 'Second Battle of Panipat',
   'c': 'Battle of Khanua',
   'd': 'Battle of Ghaghara'},
  'correct': 'b'},
 '3': {'mcq': 'Which Mughal emperor is known for his policy of conciliation and conquest towards the Rajputs?',
  'options': {'a': 'Akbar', 'b': 'Shah Jahan', 'c': 'Jahangir', 'd': 'Babur'},
  'correct': 'a'},
 '4': {'mcq': 'In which year did Shivaji have himself enthroned as an independent sovereign?',
  'options': {'a': '1650', 'b': '1674', 'c': '1680', 'd': '1700'},
  'correct': 'b'},
 '5': {'mcq': 'Which Afghan ruler defeated Humayun and expelled him from India?',
  'options': {'a': 'Shah Jahan',
   'b': 'Shah Alam',
   'c': 'Sher Shah Suri',
   'd': 'Bahadur Shah'},
 

In [26]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [27]:
quiz_table_data

[{'MCQ': 'Who founded the Maratha kingdom in 17th century India?',
  'Choices': 'a: Akbar | b: Shivaji | c: Aurangzeb | d: Humayun',
  'Correct': 'b'},
 {'MCQ': 'Which battle turned the tide in favor of the Mughal dynasty in Hindustan?',
  'Choices': 'a: First Battle of Panipat | b: Second Battle of Panipat | c: Battle of Khanua | d: Battle of Ghaghara',
  'Correct': 'b'},
 {'MCQ': 'Which Mughal emperor is known for his policy of conciliation and conquest towards the Rajputs?',
  'Choices': 'a: Akbar | b: Shah Jahan | c: Jahangir | d: Babur',
  'Correct': 'a'},
 {'MCQ': 'In which year did Shivaji have himself enthroned as an independent sovereign?',
  'Choices': 'a: 1650 | b: 1674 | c: 1680 | d: 1700',
  'Correct': 'b'},
 {'MCQ': 'Which Afghan ruler defeated Humayun and expelled him from India?',
  'Choices': 'a: Shah Jahan | b: Shah Alam | c: Sher Shah Suri | d: Bahadur Shah',
  'Correct': 'c'}]

In [28]:
quiz_table = pd.DataFrame(quiz_table_data)

In [29]:
quiz_table

,MCQ,Choices,Correct
0,Who founded the Maratha kingdom in 17th centur...,a: Akbar | b: Shivaji | c: Aurangzeb | d: Humayun,b
1,Which battle turned the tide in favor of the M...,a: First Battle of Panipat | b: Second Battle ...,b
2,Which Mughal emperor is known for his policy o...,a: Akbar | b: Shah Jahan | c: Jahangir | d: Babur,a
3,In which year did Shivaji have himself enthron...,a: 1650 | b: 1674 | c: 1680 | d: 1700,b
4,Which Afghan ruler defeated Humayun and expell...,a: Shah Jahan | b: Shah Alam | c: Sher Shah Su...,c


In [31]:
# quiz_table.to_csv("historyQ&A.csv",index=False)

In [32]:
import logging
import os
from datetime import datetime

In [33]:
datetime.now()

datetime.datetime(2024, 12, 23, 16, 11, 53, 152141)

In [34]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'12_23_2024_16_11_56'